In [1]:
%matplotlib widget

import h5py
import scipy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import time
import math
from scipy import ndimage as ndi

import skimage.morphology as morph
import skimage.measure as measure
import skimage.filters as filt
import skimage.segmentation as seg
import skimage.feature as feature
from skimage.color import label2rgb

from remove_big_objects import *

emd_file = h5py.File('/mnt/nvme1/ywlin/HTP/20210404_vaterite_alpha_0_23x23.emd', 'r')

In [2]:
# Pulling image data from EMD file
htp_images = emd_file['/data/images2048/data'][:]

# Display number of images in the stack
print(htp_images.shape)

(277, 2048, 2048)


In [3]:
# Pulling pixel size of images from metadata
metadata = emd_file['data/images2048/imageparameters']
pixelSize = metadata[10,0]

# Display pixel size in meters
print(pixelSize)

7.654511e-09


In [4]:
# Apply automated data analysis on the HTP image stack
properties = {}
labels = []

def HTP_spherefinder(im_start, im_finish, step_size):
    #start1 = time.time()
    for ii in range(im_start, im_finish, step_size):
        im_data = htp_images[ii]
        filtered_im = filt.gaussian(im_data, sigma=1.0) #Gaussian blur of input image
        
        # Generate markers (regions from objects) based on value determined by adaptive thresholding
        block_size = 511
        thresh_val = filt.threshold_local(filtered_im, block_size, offset=0)
        markers = np.zeros_like(filtered_im)
        markers[im_data < thresh_val] = 1
        markers[im_data > thresh_val] = 2
        
        # Watershed transform to fill regions of the elevation map starting from the markers
        segmentation_im = seg.watershed(filtered_im, markers)
        segmentation_im = ndi.binary_fill_holes(segmentation_im - 1)
        
        # Remove small particles and incomplete spheres
        cleaned_im0 = morph.remove_small_objects(segmentation_im, 5000)
        cleaned_im0 = remove_big_objects(cleaned_im0, 100000)
        
        # Remove objects touching image border
        cleaned_im1 = seg.clear_border(cleaned_im0, buffer_size=5)
        
        # Watershed segmentation to separate out particles that are touching and overlapping
        distance = ndi.distance_transform_edt(cleaned_im0)
        coords = feature.peak_local_max(distance, footprint=np.ones((50,50)), labels=cleaned_im0) # adjust footprint depending on feature size
        mask1 = np.zeros(distance.shape, dtype=bool)
        mask1[tuple(coords.T)] = True
        cleaned_image_markers, _ = ndi.label(mask1)
        labels_ws = seg.watershed(-distance, cleaned_image_markers, mask=cleaned_im1)
        
        # Clean data from the watershed segmentation
        labels_ws0 = morph.remove_small_objects(labels_ws, 5000)

        labels_ws1 = seg.clear_border(labels_ws0, buffer_size=5)
        labels.append(labels_ws1)
        
        # Show labeled spheres and overlay the labels onto original image data
        labeled_spheres, number = ndi.label(labels_ws1)
        
        # Measure properties from segmented regions
        properties[ii] = measure.regionprops_table(labels_ws1, properties=('area',
                                                                           'major_axis_length',
                                                                           'minor_axis_length',
                                                                           'eccentricity',
                                                                           'label',
                                                                           'centroid'))
        # Index which image in the stack the properties are from
        # properties[ii]['image_seq'] = ii
        
        #stop1 = time.time()
        #print('Time to complete: {}'.format(stop1 - start1))  
    return properties, labels

In [6]:
jj = 0 # image number in HTP image stack           
image_label_overlay = label2rgb(labels[jj], image=np.uint16(htp_images[jj]), bg_label=0)

fg, axes = plt.subplots(1, 3, figsize=(8, 3), sharey=True)
axes[0].imshow(htp_images[jj], cmap=plt.cm.gray)
axes[1].imshow(labels[jj], cmap=plt.cm.nipy_spectral, interpolation='nearest')
axes[2].imshow(image_label_overlay)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
HTP_spherefinder(0,1,1)

({0: {'area': array([18169,  9402,  6830, 13502, 14796,  7653,  7776, 10546, 32545,
          10418, 27390,  7479,  8009, 25100,  7490, 12562,  8435,  7463,
           8314, 21750,  6574, 23211, 16916, 11194,  6427,  7276, 20522,
          16447,  8889, 18771, 13216, 19717,  8828, 15722, 13665, 17513,
          15958, 15730, 10112, 13315]),
   'major_axis_length': array([166.38060965, 130.36484267,  99.23812554, 166.05140617,
          144.17652308, 117.05521326, 139.64445622, 133.27846401,
          226.97773462, 139.29910877, 196.80565789, 113.88659105,
          122.82689341, 224.04306698, 109.95028298, 146.28695972,
          118.7786832 , 120.4231608 , 106.87452086, 185.80558734,
           94.54934114, 194.53117021, 171.89154084, 151.01407633,
          120.50325815, 129.50755567, 186.44561467, 163.05174111,
          156.59619583, 186.3331046 , 143.18625637, 190.30170558,
          137.5592893 , 149.83868804, 140.81541483, 160.59916132,
          146.94889252, 151.84984659, 150.

In [9]:
# Record segmented features from multiple image frames into dict
# Note that this example only has one set of measurements that is repeated 
properties = {}
for ii in range(0,1,1):
    properties[ii] = measure.regionprops_table(labels[ii], properties=('area',
                                                               'major_axis_length',
                                                               'minor_axis_length',
                                                               'eccentricity',
                                                               'label',
                                                               'centroid'))
    properties[ii]['image_seq'] = ii

# Create dataframe with the feature properties
pd.set_option('display.max_rows', None)
df = pd.DataFrame([properties[jj] for jj in range(0,1,1)])
df = df.apply(pd.Series.explode)
df['radius']= ((df['area'] * (pixelSize ** 2)/math.pi) ** 0.5)/(1e-9)
df

,area,major_axis_length,minor_axis_length,eccentricity,label,centroid-0,centroid-1,image_seq,radius
0,18169,166.38061,141.947874,0.521663,11,291.264296,553.570037,0,582.113963
0,9402,130.364843,95.197256,0.683194,14,379.2107,363.706233,0,418.747886
0,6830,99.238126,90.252618,0.415802,15,395.8306,1860.164129,0,356.905184
0,13502,166.051406,108.197926,0.758569,16,414.489039,723.787365,0,501.812593
0,14796,144.176523,131.288498,0.413268,19,447.356515,472.398216,0,525.308787
0,7653,117.055213,87.167675,0.667431,20,658.889194,1182.279368,0,377.796874
0,7776,139.644456,84.57706,0.795724,22,777.151749,1186.237397,0,380.820773
0,10546,133.278464,101.744495,0.645929,23,802.835767,974.0293,0,443.49261
0,32545,226.977735,182.968891,0.591767,24,916.725365,556.89811,0,779.08464
0,10418,139.299109,98.407493,0.707766,25,989.362354,724.518622,0,440.792991
